# Entrega 1 - Introducción al Procesamiento del Lenguaje Natural 2018 


Verifique que su entorno de Python 3 contiene todas las bibliotecas necesarias. Importe las bibliotecas nltk y sklearn y verifique que se importan sin errores.

In [ ]:
import nltk
nltk.download('punkt')
import sklearn
import json
import os
import pprint
from random import shuffle, seed

## Lectura de datos

Observe el corpus contenido en el directorio *restaurante-review-dataset* extraído de [1]. Cargue el contenido del corpus en una variable de nombre *corpus*. Utilice como estructura de datos una lista de pares (comentario, valor), donde comentario es una string con el comentario y valor corresponde a la string "POS" o "NEG" si el comentario es positivo o negativo, respectivamente. Despliegue la cantidad de comentarios positivos y negativos.
 
[1] Dubiau, L., & Ale, J. M. (2013). Análisis de Sentimientos sobre un Corpus en Español: Experimentación con un Caso de Estudio. In Proceedings of the 14th Argentine Symposium on Artificial Intelligence, ASAI (pp. 36-47).


In [ ]:
path = "../dataset/restaurante-review-dataset/"
posPath = path + "pos/"
negPath = path + "neg/"
posFiles = os.listdir(posPath)
negFiles = os.listdir(negPath)
posJsons = [json.load(open(posPath + file)) for file in posFiles]
negJsons = [json.load(open(negPath + file)) for file in negFiles]
posCorpus = [(comentario,"POS") for lista in posJsons for comentario in lista]
negCorpus = [(comentario,"NEG") for lista in negJsons for comentario in lista]
corpus = posCorpus + negCorpus
print(str(len(posCorpus)) + " comentarios positivos y " + str(len(negCorpus)) + " comentarios negativos.")

Guarde los comentarios (sin importar el valor que tenga asignado) en una lista llamada *comentarios*. Reordene esta lista aleatoriamente. Utilice una semilla para el generador aleatorio para que sus resultados sean reproducibles. Despliegue los 3 primeros elementos de la lista *comentarios*.

In [ ]:
comentarios = [comentario for (comentario,_) in corpus]
#comentarios = [nltk.tokenize.sent_tokenize(comment, language='english') for (comment,_) in corpus]
seed(3)
shuffle(comentarios)
pprint.pprint(comentarios[0:3])

## Tokenización

Transforme cada elemento de *comentarios* en una lista de palabras utilizando las funciones de tokenización de oraciones de nltk. Almacene el resultado en una variable llamada *corpus_tokens*. Note que *corpus_tokens* es una lista donde cada elemento es a su vez una lista con los *tokens* del comentario correspondiente.

Muestre los dos últimos elementos de *corpus_tokens*.

Realice otra tokenización de los *comentarios* utilizando expresiones regulares y el módulo re de Python. En esta oportunidad cumpla con las siguientes restricciones:

- Reconocer tokens de por lo menos 3 caracteres de largo.
- No reconocer símbolos de puntuación.
- No reconocer tokens únicamente numéricos.

Almacene el resultado en una variable llamada *corpus_tokens2*.

Despliegue los dos últimos elementos de *corpus_tokens2*.

# Bag of Words

Utilice la clase CountVectorizer del módulo sklearn.feature_extraction.text para instanciar un objeto que transforme el corpus en un diccionario de las palabras más representativas de las oraciones del corpus. Investigue cada uno de los parámetros que acepta su constructor y configurelos de forma que considere adecuada. Justifique sus decisiones.

In [ ]:
tokenRegex = '[a-záéíóúñ]{3,}'
transf = sklearn.feature_extraction.text.CountVectorizer(max_df=0.5, min_df=100, ngram_range=(1,1), analyzer='word', 
                                                         lowercase=True, token_pattern=tokenRegex, stop_words=None)



- Se configura el parámetro max_df en 0.5 para no tener en cuenta palabras que aparezcan en más de la mitad de los comentarios, debido a que estas difícilmente sean representativas del mismo.

- El parámetro min_df por su parte se configura en 100 para no tener en cuenta palabras que ocurran en menos de 100 comentarios. Sabiendo que hay más de 50 mil comentarios, si una palabra aparece en menos de 100, probablemente esté mal escrita o sea un término muy específico, por lo cual no ayuda a caracterizar el comentario.

- Para tener en cuenta únicamente palabras (y no caracteres o conjuntos de palabras) se setea el parámetro analyzer en 'word' y ngram_range en (1,1).

- El parámetro lowercase se setea en True para que las mayúsculas y minúsculas sean indiferentes.

- En token_pattern se configura una expresión regular que reconoce palabras de 3 o más letras, incluyendo tildes, pero que evita reconocer por ejemplo números o caracteres especiales.

- Debido al uso de max_df se vuelve innecesario el uso de stop_words. En vez de indicar una lista de palabras a ignorar, se ignoran las palabras que aparecen con demasiada frecuencia.

Entrene la transformación e imprima las palabras resultantes en el diccionario. Imprima además la cantidad de palabras obtenidas en el diccionario.

In [ ]:
transf.fit(comentarios)

In [ ]:
diccionario = transf.get_feature_names()
print(diccionario)
print("El diccionario tiene " + str(len(diccionario)) + " palabras.")

Transforme todos los *comentarios* en vectores utilizando la transformación creada. Guarde las transformaciones en una lista llamada 'vectores':

In [ ]:
vectores = transf.transform(comentarios)

print(len(vectores.toarray()[0]))

## Similitud

Escriba una función calcular_similitud(vect1, vect2) que dados dos vectores, devuelva un número en el rango [0, 1] donde 1 significa que los vectores son idénticos y 0 que son completamente diferentes. Puede utilizar medidas como la implementada en la función cosine del módulo scipy.spatial.distance u otra que considere adecuada.

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine as dist 

def calcular_similitud(vec1, vec2):
  normaVec1 = np.linalg.norm(vec1)
  normaVec2 = np.linalg.norm(vec2)
  if (dist.__name__ in ["cosine", "braycurtis"]) and (normaVec1 == 0 or normaVec2 == 0):
    return 0
  # Como las distancias de coseno y Bray-Curtis
  distancia = dist(vec1, vec2)
  return 1/(1 + distancia) 

Como las funciones de distancia son crecientes y además no garantizan un valor entre 0 y 1 (a excepción de las distancias de coseno y bray-curtis), la función devuelve `1/(1 + distancia(v1, v2))` para obtener el valor de similitud.

Vamos a definir los siguientes dos diccionarios:

In [ ]:
similar = {}
similitud = {}

El primer diccionario guardará para cada oración, el índice de la oración más similar a ella. Esto es, similar[i] = j indica que la j-esima oración, es la oración más similar a la i-esima oración.
El segundo diccionario guardará el valor de similitud correspondiente. Si similiar[i] = j, entonces similitud[i] = calcular_similitud(oracion[i], oracion[j]).
Implemente una fución llamada calcular_similitudes(vectores, similar, similitud) que dada la lista de vectores, construya los diccionarios similar y similitud:

In [ ]:
from heapq import heappush

def calcular_similitudes(vectores, similar, similitud):
  cantVectores = len(vectores)
  similitudes = [[] for _ in range(cantVectores)]
  for i in range(cantVectores):
    for j in range(i+1, cantVectores):
        dist = calcular_similitud(vectores[i], vectores[j])
        heappush(similitudes[i], (dist, j))
        heappush(similitudes[j], (dist, i))
    maximo = max(similitudes[i])
    similitud[i] = maximo[0]
    similar[i] = maximo[1]

En esta función se utiliza una cola de prioridad (modelada como un heap) para almacenar una dupla con los valores de similitud y el comentario al que corresponde dicho valor de manera ordenada. Cuando se terminan de computar las distancias de un vector con el resto de los vectores, se obtiene el valor máximo y se inserta en valor de similitud en `similitud` y el comentario al que corresponde en `similar`.

Implemente una función llamada *imprimir_similares* que imprima los primeros N pares de comentarios más similares del corpus. Utilice un valor que considere adecuado para N (ej. N=50).

In [ ]:
from operator import itemgetter

def imprimir_similares(comentarios, similar, similitud, cantComentarios=50):
  simlitudesEnumeradas = list(zip(similar.keys(), similitud.values()))
  # Para conservar a que comentario corresponde cada valor
  similitudesOrdenadas = sorted(simlitudesEnumeradas, key=itemgetter(1)) 
  # Se ordenan las similitudes
  comentariosImpresos = 0
  i = len(similitud) - 1
  impresos = [False for _ in range(len(similitud))]
  while comentariosImpresos < cantComentarios and i < len(comentarios):
    posComentario = similitudesOrdenadas[i][0]
    posComentarioSimilar = similar[posComentario]
    if not impresos[posComentario] or not impresos[posComentarioSimilar]:
        print("El comentario nro. {}:\n\n{}\n".format(posComentario, comentarios[posComentario])) 
        print("Es similar al nro. {}:\n\n{}\n".format(posComentarioSimilar, comentarios[posComentarioSimilar]))
        print("Con similitud {}\n\n".format(round(similitud[posComentario], 3)))
        print("*************************************\n") 
        impresos[posComentario] = True
        impresos[posComentarioSimilar] = True
    comentariosImpresos += 1
    i -= 1

Para no imprimir el mismo par de comentario dos veces, se guarda en `impresos` los comentarios que ya se imprimieron. Si un par ya fue impreso, no se vuelve a imprimir. Esto puede suceder cuando dos comentarios son similares maximalmente entre ellos.

Ejecute las funciones calcular_similitudes e imprimir_similares. Utilice una porción del corpus que considere adecuada para que ejecute en un tiempo razonable (ej. los primeros 1000 comentarios). Despliegue los resultados y el tiempo transcurrido. Si lo desea puede incluir resultados en un bloque de texto al considerar una porción mayor. De ser así incluya la cantidad de elementos considerados y el tiempo transcurrido. 

In [ ]:
from timeit import default_timer as timer

tComienzo = timer()
calcular_similitudes(vectores.toarray()[:1000], similar, similitud)
imprimir_similares(comentarios, similar, similitud)
tFinal = timer()
print("Demoró {} segundos.".format(round(tFinal - tComienzo, 3)))

En las pruebas realizadas anteriormente, se transforma la matriz dispersa en un arreglo de vectores. A continuación, mostramos los resultados utilizando operaciones para trabajar con matrices dispersas.

Primero, redefinimos las funciones calcular_similitud y calcular_similitudes para que trabajen con matrices dispersas:

In [ ]:
from sklearn.metrics.pairwise import cosine_distances as dist
# Se puede utilizar algunas de las métricas incluidas en 'sklearn.matrix.pairwise' 
# las cuales son: cosine_distances, euclidean_distances y manhattan_distances.

def calcular_similitud_disp(vec1, vec2):
  return 1/(1 + dist(vec1, vec2)[0][0]) 

In [ ]:
def calcular_similitudes_disp(vectores, similar, similitud):
  cantVectores = vectores.shape[0]
  similitudes = [[] for _ in range(cantVectores)]
  for i in range(cantVectores):
    for j in range(i+1, cantVectores):
        dist = calcular_similitud_disp(vectores[i], vectores[j])
        heappush(similitudes[i], (dist, j))
        heappush(similitudes[j], (dist, i))
    maximo = max(similitudes[i])
    similitud[i] = maximo[0]
    similar[i] = maximo[1]

Luego, ejecutamos las mismas funciones que se utilizaron anteriormente para calcular los diccionarios similar y similitud
e imprimir los comentarios más similares:

In [ ]:
tComienzo = timer()
calcular_similitudes_disp(vectores[:1000], similar, similitud)
imprimir_similares(comentarios, similar, similitud)
tFinal = timer()
print("Demoró {} segundos.".format(round(tFinal - tComienzo, 3)))

Los resultados obtenidos son exáctamente los mismos que los obtenidos anteriormente. Se puede ver que al utilizar las funciones que contemplan  el manejo de matrices dispersas, la eficiencia computacional empeora considerablemente. Esto puede suceder ya que las funciones de distancia provistas por la librería Scikit Learn no son muy eficientes, incluso para el manejo de matrices dispersas. 

Analice los resultados obtenidos. ¿Que observa? ¿En qué opina que se podría mejorar?

Utilizando el parámetro 'max_df' en 0.50, pudimos ver que los mejores resultados se obtuvieron con oraciones cortas. Esto se puede explicar observando que las palabras son muy comunes en el tipo de comentarios que se está analizando (excelente, muy, bueno, buena, comida, etc.). En el caso de oraciones largas, es muy probable que, aunque puedan contener estas palabras, el resto de las palabras no aparezcan comúnmente, por lo que aunque el valor de similitud sea alto los comentarios no son similares a la vista.

Otro aspecto a considerar es que es probable que los comentarios contengan faltas de ortografía o errores de tipeo. Si utilizamos nociones de distancia entre vectores, estas palabras no se toman como iguales, lo que puede ser deseable para computar un valor de similitud entre palabaras o textos. En estos casos, es deseable utilizar una medida de distancia entre strings, como la distancia de Levenshtein o Jaro-Winkler.

## Bag of Words (con stemming)

Utilice el parámetro 'tokenizer' de la clase CountVectorizer y conjuntamente la clase SpanishStemmer del módulo nltk.stem.snowball. Repita los experimentos realizados en la parte anterior utilizando el stemmer. Realice las comparaciones que considere pertinentes.

Al igual que en la parte anterior muestre los resultados para una porción del corpus y si lo desea incluya los resultados para una porción mayor. Despliegue el tiempo transcurrido.

Justifique las decisiones tomadas y realice los comentarios que considere adecuados. Analice los resultados obtenidos. Compare los resultados con los obtenidos anteriormente. Comente los problemas que detecte en este enfoque.